In [ ]:
import ipywidgets as widgets
import xarray
from io import BytesIO
import typing
import numbers
import subprocess
from shlex import quote
import requests

In [ ]:
url_loc = widgets.Text(placeholder="ERDDAP NetCDF URL")
display(url_loc)

In [ ]:
if uploader.value[0].type == "application/x-netcdf":
    ds = xarray.open_dataset(BytesIO(uploader.value[0].content.tobytes()))

In [ ]:
url = url_loc.value
print(url)
# whether we need to store the dataset to disk before processing,
# usually due to needing to fetch a netCDF file from ERDDAP due to
# not presenting an OPeNDAP interface
store_remote = False
try:
    with warnings.catch_warnings() as w:
        warnings.simplefilter("ignore")
        ds = xarray.open_dataset(url)
        
# handle ERDDAP, which doesn't use an OPeNDAP interface for TableDAP
except OSError:
    req = requests.get(url, timeout=30)
    ds = xarray.open_dataset(BytesIO(req.content))
    store_remote = True

In [ ]:
[k for k in ds.coords.keys()]

In [ ]:
import ipywidgets as widgets

time_variable_selection = widgets.Dropdown(
    options=[k for k in ds.variables.keys()],
    description='Time variable:',
    disabled=False,
)
display(time_variable_selection)

In [ ]:
input_variable_selection = widgets.SelectMultiple(
    #options=['1', '2', '3'],
    options=[k for k in ds.variables.keys()],
    #value='2',
    description='Input variable(s):',
    disabled=False,
)
display(input_variable_selection)

In [ ]:
x_variable_selection = widgets.Dropdown(
    options=[k for k in ds.variables.keys()],
    description='x/lon variable:',
    disabled=False,
)
display(x_variable_selection)

In [ ]:
y_variable_selection = widgets.Dropdown(
    options=[k for k in ds.variables.keys()],
    description='y/lat variable:',
    disabled=False,
)
display(y_variable_selection)

In [ ]:
depth_variable_selection = widgets.Dropdown(
    options=[None] + [k for k in ds.variables.keys()],
    description='Depth/z variable:',
    disabled=False,
)
display(depth_variable_selection)

In [ ]:
time_variable = ds.variables[time_selection.value]
input_variable_dict = {var_name: ds.variables[var_name] for var_name in input_variable_selection.value}
x_variable = ds.variables[x_variable_selection.value]
y_variable = ds.variables[y_variable_selection.value]
depth_variable = None if depth_variable_selection.value is None else ds.variables[depth_variable_selection.value]

In [ ]:
# can't seem to import due to dependency solving

import cf_units

In [ ]:
from ioos_qc import qartod

In [ ]:
qartod.location_test.__annotations__

In [ ]:
import inflection
import warnings
import numbers


def widget_factory_type(variable_name, function):
    widget_controls = []
    annotations = function.__annotations__
    # TODO: for when proper widget is found for bbox selection for location test
    #bbox_type = typing.Tuple[numbers.Real, numbers.Real, numbers.Real, numbers.Real]
    for annotation_name, type_signature in annotations.items():
        # data variables should be skipped as they aren't tunable for QARTOD test parameters
        if annotation_name in {"inp", "tinp", "lon", "lat", "zinp", "method", "return", "type_signature", "check_type"}:
            continue
            
        match type_signature:
            case t if t is numbers.Real or t is float:
                widget_controls.append(widgets.FloatSlider(value=0, min=-500, max=500,
                                                           description=annotation_name,
                                                           name=f"{variable_name} {function} {annotation_name}"))
            case t if t is int:
                widget_controls.append(widgets.IntSlider(value=0, min=0, max=100,
                                                    description=annotation_name,
                                                    name=f"{variable_name} {function} {annotation_name}"))
            case t if t is typing.Tuple[numbers.Real, numbers.Real]:
                widget_controls.append(widgets.FloatRangeSlider(value=[-100, 100], min=-500, max=500,
                                                                description=annotation_name,
                                                                name=f"{variable_name} {function} {annotation_name}"))             
            # four numbers could only be geospatial bounding box for location test
            # TODO: display with interact, etc
            case t if t is typing.Tuple[numbers.Real, numbers.Real, numbers.Real, numbers.Real]:
                widget_controls.append(widgets.VBox([widgets.FloatRangeSlider(value=[0, 1], min=-180, max=180,
                                                           description="Longitude Bounds"),
                                  widgets.FloatRangeSlider(value=[0, 1], min=-90, max=90,
                                                           description="Latitude Bounds")],
                                                    name=f"{variable_name} {function} {annotation_name}"))
            case _:
                warnings.warn(f"Parameter {annotation_name} with type signature {type_signature} unmatched, skipping...")
                continue
    checkbox = widgets.Checkbox(False, description="Disabled", indent=False)
    for control in widget_controls:
        widgets.link((checkbox, 'value'), (control, 'disabled'))
    return widgets.VBox([checkbox, widgets.VBox(widget_controls)])

In [ ]:
def show_controls(variable_name, function): 
    print(inflection.titleize(function.__name__))
    display(widget_factory_type(variable_name, function))

In [ ]:
import inspect
from ioos_qc import qartod

test_dict = {}
qartod_tests = inspect.getmembers(qartod, lambda o: inspect.isfunction(o) and o.__name__.endswith("_test"))
for input_variable_name, input_variable in input_variable_dict.items():
    test_dict[input_variable_name] = {}
    print(input_variable_name)
    for test_name, qartod_test in qartod_tests:
        # don't know how to represent climatology widgets
        if test_name in {"climatology_test", "location_test"}:
            continue
        test_dict[input_variable_name][test_name] = widget_factory_type(input_variable_name, qartod_test)
        print(inflection.titleize(qartod_test.__name__))
        show_controls(input_variable_name, qartod_test)
display(widgets.Checkbox(True, description="Run CF Compliance Check", indent=False))
display(widgets.Button(description="Submit"))

In [ ]:
def generate_qc_json_config(widget_conf):
    qc_conf_dict = {}
    for variable_name in widget_conf:
        qc_conf_dict[variable_name] = {}
        qartod_conf = qc_conf_dict[variable_name]["qartod"] = {}
        for test_name in widget_conf[variable_name]:
            # if test is disabled, skip
            if test_dict[variable_name][test_name].children[0].value:
                continue
            qartod_conf[test_name] = {}
            test_widgets = test_dict[variable_name][test_name].children[1].children
            for test_widget in test_widgets:
                qartod_conf[test_name][test_widget.description] = test_widget.value
    return qc_conf_dict

qc_config = generate_qc_json_config(test_dict)
            

In [ ]:
cmds = ["argo", "submit",
                "-p", f"file-url={quote(url)}",
                "-p", f"qc-config={quote(json.dumps(qc_config))}",
                "-p", f"input-var-selection={quote(json.dumps(input_variable_selection.value))}",
                "-p", f"x-var-name={quote(x_variable_selection.value)}",
                "-p", f"y-var-name={quote(y_variable_selection.value)}",
                "-p", f"z-var-name={quote(depth_variable_selection.value)}",
                "-p", f"t-var-name={quote(time_variable_selection.value)}", 
                "curl_file.yaml"]

subprocess.run(cmds)